In [1]:
import numpy as np
import qutip as qt
import cvxpy as cp

In [2]:
def create_random_choi(dim_in, dim_out):
    """
    Create a random valid Choi matrix using QuTiP's quantum info tools.
    """
    dim_in, dim_out = int(dim_in), int(dim_out)
    
    # Create random Kraus operators
    num_kraus = min(dim_in, dim_out)
    kraus_list = []
    
    for _ in range(num_kraus):
        # Random complex matrix
        K = qt.rand_unitary(dim_out) * (1.0/num_kraus)
        kraus_list.append(K)
    
    # Create the superoperator from Kraus operators
    superop = qt.kraus_to_super(kraus_list)
    
    # Convert to Choi matrix
    choi = qt.to_choi(superop)
    return choi

In [3]:
create_random_choi(2, 2)

Quantum object: dims=[[[2], [2]], [[2], [2]]], shape=(4, 4), type='super', dtype=Dense, isherm=True, superrep='choi'
Qobj data =
[[ 0.44160585-1.06003126e-19j  0.06919115+5.02969833e-02j
   0.1290835 -4.32111853e-02j -0.02635382-1.71674838e-01j]
 [ 0.06919115-5.02969833e-02j  0.05839415-7.68175176e-19j
   0.0378506 -3.48505807e-02j -0.1290835 +4.32111853e-02j]
 [ 0.1290835 +4.32111853e-02j  0.0378506 +3.48505807e-02j
   0.05839415-8.38583451e-19j -0.06919115-5.02969833e-02j]
 [-0.02635382+1.71674838e-01j -0.1290835 -4.32111853e-02j
  -0.06919115+5.02969833e-02j  0.44160585+1.56216640e-18j]]

In [4]:
def create_identity_choi(dim):
    """
    Create Choi matrix for the identity channel on dim-dimensional space.
    """
    id_super = qt.superop_reps.identity(dim)
    return qt.to_choi(id_super)



In [14]:
def apply_partial_transpose_cvxpy(matrix, dims, subsystems):
    """
    Apply partial transpose to CVXPY variable using index manipulation.
    
    Parameters:
    matrix: CVXPY variable to transpose
    dims: list of dimensions for each subsystem [dim1, dim2, ...]
    subsystems: list of subsystem indices to transpose (0-based)
    
    Returns:
    CVXPY variable with partial transpose applied
    """
    total_dim = np.prod(dims)
    if matrix.shape != (total_dim, total_dim):
        raise ValueError(f"Matrix shape {matrix.shape} doesn't match total dim {total_dim}")
    
    # Reshape to tensor form
    reshaped = cp.reshape(matrix, dims + dims)
    
    # Apply transpose to specified subsystems
    # For partial transpose, we transpose the specified subsystems in both input and output
    transpose_indices = list(range(len(dims)))
    for sub in subsystems:
        if sub < len(dims):
            # Swap input and output indices for this subsystem
            transpose_indices[sub] = sub + len(dims)
            transpose_indices[sub + len(dims)] = sub
    
    # Apply the permutation
    transposed = cp.transpose(reshaped, transpose_indices)
    
    # Reshape back to matrix form
    return cp.reshape(transposed, (total_dim, total_dim))

def superchannel_sdp(J_Phi, J_Psi, dims):
    """
    Solve the superchannel SDP for channel conversion.
    """
    # Extract dimensions as integers
    dim_X = int(dims['X'])
    dim_Y = int(dims['Y']) 
    dim_Xp = int(dims['Xp'])
    dim_Yp = int(dims['Yp'])
    
    # Total dimensions
    dim_total = dim_Yp * dim_Xp * dim_Y * dim_X
    
    # Convert QuTiP objects to numpy arrays
    J_Phi_np = J_Phi.full()
    J_Psi_np = J_Psi.full()
    
    # Create SDP variables
    J_Theta = cp.Variable((dim_total, dim_total), complex=True)
    J1 = cp.Variable((dim_Xp * dim_Y * dim_X, dim_Xp * dim_Y * dim_X), complex=True)
    
    # Identity matrices
    I_Yp = np.eye(dim_Yp)
    I_Xp = np.eye(dim_Xp)
    I_X = np.eye(dim_X)
    
    # Structural constraint: Tr_Y' J(Θ) = I_X' ⊗ J1
    J_Theta_reshaped = cp.reshape(J_Theta, (dim_Yp, dim_Xp, dim_Y, dim_X, dim_Yp, dim_Xp, dim_Y, dim_X))
    
    # Partial trace over Y' (sum over first index with matching output index)
    J1_from_trace_terms = []
    for i in range(dim_Yp):
        term = J_Theta_reshaped[i, :, :, :, i, :, :, :]
        J1_from_trace_terms.append(cp.reshape(term, (dim_Xp * dim_Y * dim_X, dim_Xp * dim_Y * dim_X)))
    
    J1_from_trace_flat = cp.sum(J1_from_trace_terms)
    
    # Structural constraint: Tr_Y J1 = I_X
    J1_reshaped = cp.reshape(J1, (dim_Xp, dim_Y, dim_X, dim_Xp, dim_Y, dim_X))
    tr_Y_J1_terms = []
    for i in range(dim_Y):
        term = J1_reshaped[:, i, :, :, i, :]
        tr_Y_J1_terms.append(cp.reshape(term, (dim_Xp * dim_X, dim_Xp * dim_X)))
    
    tr_Y_J1_flat = cp.sum(tr_Y_J1_terms)
    
    # Channel conversion constraint using proper partial transpose
    # We need to compute: Tr_{Y,X}[(J(Θ)^{T_{X,Y}} (I_Y' ⊗ J(Φ))] = J(Ψ)
    
    # Apply partial transpose on X,Y subsystems using our custom function
    # J(Θ) has dimensions [Y', X', Y, X, Y', X', Y, X]
    # We want to transpose X and Y subsystems (indices 2,3 and 6,7 in 0-based indexing)
    J_Theta_T_flat = apply_partial_transpose_cvxpy(
        J_Theta, 
        [dim_Yp, dim_Xp, dim_Y, dim_X],  # Input dimensions
        [2, 3]  # Transpose on Y,X subsystems (0-based: 2=Y, 3=X)
    )
    
    # Create I_Y' ⊗ J(Φ) with proper dimensions
    # J(Φ) has shape (dim_Y * dim_X, dim_Y * dim_X)
    # I_Y' ⊗ J(Φ) should have shape (dim_Yp * dim_Y * dim_X, dim_Yp * dim_Y * dim_X)
    # But wait - we need to match the dimensions of J_Theta_T which is (dim_Yp * dim_Xp * dim_Y * dim_X, ...)
    # Actually, we need: I_Y' ⊗ J(Φ) with dimensions matching the appropriate subsystems
    
    # Let's think about this carefully:
    # J(Θ) operates on: Y' ⊗ X' ⊗ Y ⊗ X → Y' ⊗ X' ⊗ Y ⊗ X
    # After partial transpose on Y,X: dimensions stay the same
    # I_Y' ⊗ J(Φ) should operate on the same space
    
    # The correct approach: I_Y' ⊗ I_X' ⊗ J(Φ) but we only have Y' and X' in the output
    # Actually, we need: I_{Y'} ⊗ (J(Φ) ⊗ I_{X'}) but properly arranged
    
    # Let's create the correct tensor product:
    # We want an operator that acts as identity on Y',X' and applies J(Φ) on Y,X
    I_YpXp = np.eye(dim_Yp * dim_Xp)
    correct_tensor_product = np.kron(I_YpXp, J_Phi_np)
    
    # Now matrix multiplication should work
    product = J_Theta_T_flat @ correct_tensor_product
    
    # Reshape for partial trace over Y,X
    product_reshaped = cp.reshape(product, (dim_Yp, dim_Xp, dim_Y, dim_X, dim_Yp, dim_Xp, dim_Y, dim_X))
    
    # Partial trace over Y,X (sum over input and output indices 2,3 and 6,7)
    converted_terms = []
    for i in range(dim_Y):
        for j in range(dim_X):
            term = product_reshaped[:, :, i, j, :, :, i, j]
            converted_terms.append(cp.reshape(term, (dim_Yp * dim_Xp, dim_Yp * dim_Xp)))
    
    converted_channel_flat = cp.sum(converted_terms)
    
    # Constraints
    constraints = [
        J_Theta >> 0,
        J1 >> 0,
        J1_from_trace_flat == cp.kron(I_Xp, J1),
        tr_Y_J1_flat == I_X,
        converted_channel_flat == J_Psi_np
    ]
    
    # Objective: minimize trace norm difference (approximated)
    H = np.eye(dim_total)
    objective = cp.Minimize(cp.real(cp.trace(H @ J_Theta)))
    
    # Solve SDP
    prob = cp.Problem(objective, constraints)
    prob.solve()
    
    return {
        'status': prob.status,
        'optimal_value': prob.value,
        'J_Theta_opt': J_Theta.value,
        'J1_opt': J1.value
    }


In [15]:
def verify_superchannel_constraints(J_Theta_opt, dims, tol=1e-6):
    """
    Verify that the superchannel Choi operator satisfies all constraints using QuTiP.
    """
    dim_Yp = int(dims['Yp'])
    dim_Xp = int(dims['Xp'])
    dim_Y = int(dims['Y'])
    dim_X = int(dims['X'])
    
    # Convert to QuTiP object
    J_Theta_qt = qt.Qobj(J_Theta_opt, dims=[[dim_Yp, dim_Xp, dim_Y, dim_X], [dim_Yp, dim_Xp, dim_Y, dim_X]])
    
    # Check positivity
    eigvals = J_Theta_qt.eigenstates()[0]
    is_positive = np.all(np.real(eigvals) >= -tol)
    
    # Check Tr_Y' J(Θ) structure using QuTiP partial trace
    J1_from_trace_qt = qt.ptrace(J_Theta_qt, [1, 2, 3])  # Trace over Y', keep X', Y, X
    
    # Check that the result has the form I_X' ⊗ J1
    # We can verify by checking that partial trace over X' gives proportional identity
    J1_test = J1_from_trace_qt.full().reshape(dim_Xp, dim_Y, dim_X, dim_Xp, dim_Y, dim_X)
    
    # Check Tr_Y J1 = I_X using QuTiP
    J1_test_qt = qt.Qobj(J1_from_trace_qt.full(), dims=[[dim_Xp, dim_Y, dim_X], [dim_Xp, dim_Y, dim_X]])
    tr_Y_J1_qt = qt.ptrace(J1_test_qt, [0, 2])  # Trace over Y, keep X', X
    
    is_identity = np.allclose(tr_Y_J1_qt.full(), np.eye(dim_Xp * dim_X), atol=tol)
    
    # Verify channel conversion using QuTiP partial transpose
    J_Phi_test = create_identity_choi(dims['X'])  # Test with identity channel
    J_Psi_test = create_identity_choi(dims['Xp'])
    
    # Create test superchannel
    J_Theta_test_qt = qt.Qobj(J_Theta_opt, dims=[[dim_Yp, dim_Xp, dim_Y, dim_X], [dim_Yp, dim_Xp, dim_Y, dim_X]])
    
    # Apply partial transpose using QuTiP
    J_Theta_T_test_qt = qt.partial_transpose(J_Theta_test_qt, [2, 3])  # Transpose on Y,X subsystems
    
    # Compute converted channel: Tr_{Y,X}[(J_Theta^T)(I_Y' ⊗ J_Phi)]
    I_Yp_qt = qt.qeye([dim_Yp])
    I_Yp_tensor_J_Phi_qt = qt.tensor(I_Yp_qt, J_Phi_test)
    
    product_qt = J_Theta_T_test_qt * I_Yp_tensor_J_Phi_qt
    converted_qt = qt.ptrace(product_qt, [0, 1])  # Trace over Y,X, keep Y', X'
    
    conversion_valid = np.allclose(converted_qt.full(), J_Psi_test.full(), atol=tol)
    
    return {
        'is_positive': is_positive,
        'tr_Y_J1_is_identity': is_identity,
        'min_eigenvalue': np.min(np.real(eigvals)),
        'channel_conversion_valid': conversion_valid
    }

In [16]:
def test_with_simple_channels():
    """
    Test the SDP with simple identity channels.
    """
    print("Testing with identity channels...")
    
    dims = {'X': 2, 'Y': 2, 'Xp': 2, 'Yp': 2}
    
    # Create identity channels using QuTiP
    J_Phi = create_identity_choi(dims['X'])
    J_Psi = create_identity_choi(dims['Xp'])
    
    print(f"Input channel dimensions: {J_Phi.dims}")
    print(f"Target channel dimensions: {J_Psi.dims}")
    
    print("Solving SDP for identity channel conversion...")
    result = superchannel_sdp(J_Phi, J_Psi, dims)
    
    print(f"SDP status: {result['status']}")
    if result['optimal_value'] is not None:
        print(f"Optimal value: {result['optimal_value']:.6f}")
    
    if result['status'] in ['optimal', 'optimal_inaccurate'] and result['J_Theta_opt'] is not None:
        verification = verify_superchannel_constraints(result['J_Theta_opt'], dims)
        print(f"J(Θ) is positive: {verification['is_positive']}")
        print(f"Tr_Y J1 = I_X: {verification['tr_Y_J1_is_identity']}")
        print(f"Minimum eigenvalue: {verification['min_eigenvalue']:.2e}")
        print(f"Channel conversion valid: {verification['channel_conversion_valid']}")
    else:
        print("Could not verify solution - SDP may not have converged properly")

In [17]:
def test_with_random_channels():
    """
    Test the SDP with random channels.
    """
    print("\n" + "="*50)
    print("Testing with random channels...")
    
    dims = {'X': 2, 'Y': 2, 'Xp': 2, 'Yp': 2}
    
    # Create random channels
    print("Creating random input and target channels...")
    J_Phi = create_random_choi(dims['X'], dims['Y'])
    J_Psi = create_random_choi(dims['Xp'], dims['Yp'])
    
    print(f"Input channel dimensions: {J_Phi.dims}")
    print(f"Target channel dimensions: {J_Psi.dims}")
    
    # Verify channels are valid
    print("\nVerifying input channels:")
    print(f"J(Φ) is positive: {J_Phi.iscp}")
    print(f"J(Ψ) is positive: {J_Psi.iscp}")
    
    # Solve SDP
    print("\nSolving superchannel SDP...")
    result = superchannel_sdp(J_Phi, J_Psi, dims)
    
    print(f"SDP status: {result['status']}")
    if result['optimal_value'] is not None:
        print(f"Optimal value: {result['optimal_value']:.6f}")
    
    if result['status'] in ['optimal', 'optimal_inaccurate'] and result['J_Theta_opt'] is not None:
        verification = verify_superchannel_constraints(result['J_Theta_opt'], dims)
        print(f"J(Θ) is positive: {verification['is_positive']}")
        print(f"Tr_Y J1 = I_X: {verification['tr_Y_J1_is_identity']}")
        print(f"Minimum eigenvalue: {verification['min_eigenvalue']:.2e}")
        print(f"Channel conversion valid: {verification['channel_conversion_valid']}")
    else:
        print("SDP did not converge to optimal solution")


In [18]:
test_with_simple_channels()

Testing with identity channels...
Input channel dimensions: [[[2], [2]], [[2], [2]]]
Target channel dimensions: [[[2], [2]], [[2], [2]]]
Solving SDP for identity channel conversion...


IndexError: list assignment index out of range